# Ch. 2. Resource Managers

## I. Interacting With the Operating System
Programming languages are nice for carrying out computations based on input data. They have useful constructs like `for` loops, `if` statements, and module systems so that you can reuse code easily. However, for certain tasks, what's built in to the programming language is simply not enough. For situations like this, programming languages usually allow explicit interaction with the underlying operating system. In Python, this is handled by the `os` module. The following graphic helps to describe how the underlying OS behaves and how we can interact with it:
![OS Architecture Diagram](https://docs.mesosphere.com/1.11/img/dcos-architecture-layers.png)

The code you have been running in these notebooks fits into the "services" section of the software layer, and our OS interaction will allow us to request data and provide instructions to the objects in the "platform" layer.


It is important to remember that Python code is meant to be portable, and you have to be careful when using the `os` module to ensure that your code can still be run on your target systems. This course is designed for \*nix  systems (including macOS and BSD), and certain code written in examples will not run on windows systems, especially code from this chapter. 

### Example 2.1a - Basic Filesystem Interaction
In this example, we will interact with the filesystem around us. The module `os` contains a lot of useful functionality for interacting with the filesystem, networking layers, and other OS-specific methods. For more information see the [`os` docs](https://docs.python.org/3/library/os.html).

In [ ]:
# Ex. 2.1a - Filesystem interaction 

import os

# We can list directories
print(os.listdir('/home/users/glick/intro-to-hpc'))

# We can make directories
print(os.mkdir('/home/users/glick/intro-to-hpc/dira'))

# List again to see the new directory
print(os.listdir('/home/users/glick/intro-to-hpc'))

# We can removie directories too
print(os.rmdir('/home/users/glick/intro-to-hpc/dira'))

# Finally, list again to see the new directory
print(os.listdir('/home/users/glick/intro-to-hpc'))


# Using open(), we can read and write files
with open("/home/users/glick/intro-to-hpc/data/message.txt", "w") as file:
    file.write("Super Secret Message")
    
with open("/home/users/glick/intro-to-hpc/data/message.txt", "r") as file:
    print(file.read())
    
# Delete our secret message
os.remove("/home/users/glick/intro-to-hpc/data/message.txt")

### Example 2.1b - Opening Processes
For things as universal as filesystem interaction, `os` has built-in functions to help you out. However, there are some things that the Python developers simply can't prepare for. Let's say you want to compile some code, run it, and print the output, all from within Python. This is simple with the command line, and Python's `os` module provides a way to run system commands through the command line, through a function called `os.popen()`. Please remember that this can be unsafe if you allow malicious actors to access your computer, so only use the `os.popen()` function when necessary.

In [ ]:
# Opening Processes
# There is some code in /home/users/glick/intro-to-hpc/data/hello.c

import os

# Compile the code - There should be no output
process = os.popen("cc /home/users/glick/intro-to-hpc/data/hello.c -o /home/users/glick/intro-to-hpc/data/hello.out")
print(process.read())

# Run the code
process = os.popen("/home/users/glick/intro-to-hpc/data/hello.out")
print(process.read())

## II. What is a Resource Manager?

### Example 2.2 - Submitting and Monitoring a Simple Resource Manager Job

## III. Programmatic Interaction

### Example 2.3 - Lots of Jobs!

## IV. Basic Dataflows

### Example 2.4 - Array Job Plus Summary Statistics

## Exercise 2. Array Dataflow with `sys`